FLIGHT DELAY SCORE GENERATOR

In [ ]:
import pandas as pd
import numpy as np


def calculate_flight_difficulty_score(flight_data_path, bag_data_path, pnr_flight_data_path, pnr_remark_data_path,
                                      use_random_conn_time=True, difficult_q=0.8, easy_q=0.2):

    # Step 1: Load data
    df_flights = pd.read_csv(flight_data_path)
    df_bags = pd.read_csv(bag_data_path)
    df_pnr = pd.read_csv(pnr_flight_data_path)
    df_remarks = pd.read_csv(pnr_remark_data_path)

    dep_dt = pd.to_datetime(df_flights.get('scheduled_departure_datetime_local', pd.NaT), errors='coerce')
    df_flights['departure_hour'] = dep_dt.dt.hour


    # Step 2: Fix remarks with date by merging with PNR lookup
    pnr_lookup = df_pnr[['record_locator', 'flight_number', 'scheduled_departure_date_local']].drop_duplicates()
    df_remarks_with_date = pd.merge(
        df_remarks, pnr_lookup, on=['record_locator', 'flight_number'], how='left'
    )


    # Step 3: Aggregate bag data
    df_bags = df_bags.copy()
    if use_random_conn_time:
        np.random.seed(42)
        transfer_idx = df_bags.index[df_bags['bag_type'] == 'Transfer']
        df_bags.loc[transfer_idx, 'connection_time'] = np.random.randint(10, 120, size=len(transfer_idx))


    df_bags['is_hot_transfer'] = (df_bags['bag_type'] == 'Transfer') & (df_bags['connection_time'] < 30)
    df_bags['is_other_transfer'] = (df_bags['bag_type'] == 'Transfer') & (df_bags['connection_time'] >= 30)


    bag_counts = df_bags.groupby(['flight_number', 'scheduled_departure_date_local']).agg(
        total_checked_bags=('bag_tag_unique_number', 'count'),
        hot_transfer_bags=('is_hot_transfer', 'sum'),
        other_transfer_bags=('is_other_transfer', 'sum')
    ).reset_index()


    # Step 4: Aggregate PNR and SSR data
    pnr_counts = df_pnr.groupby(['flight_number', 'scheduled_departure_date_local']) \
                       .agg(total_pax=('total_pax', 'sum')).reset_index()


    ssr_counts = df_remarks_with_date.groupby(['flight_number', 'scheduled_departure_date_local']) \
                                     .agg(ssr_count=('special_service_request', 'count')).reset_index()


    # Step 5: Merge to flight level
    merge_keys = ['flight_number', 'scheduled_departure_date_local']
    df = pd.merge(df_flights, bag_counts, on=merge_keys, how='left')
    df = pd.merge(df, pnr_counts, on=merge_keys, how='left')
    df = pd.merge(df, ssr_counts, on=merge_keys, how='left')


    # Fill missing aggregates
    for col in ['total_checked_bags', 'hot_transfer_bags', 'other_transfer_bags', 'total_pax', 'ssr_count']:
        if col in df.columns:
            df[col] = df[col].fillna(0)


    # Step 6: Feature engineering
    # Turn pressure: smaller or negative scheduled ground time vs minimum implies pressure; invert sign to keep higher worse
    df['turnaround_pressure'] = df['minimum_turn_minutes'] - df['scheduled_ground_time_minutes']


    # Bag complexity
    df['baggage_complexity_score'] = (df['hot_transfer_bags'] * 2) + df['other_transfer_bags']


    # Passenger service load
    df['passenger_service_load'] = df['ssr_count']


    # Congestion factor: robust to missing hours
    # Map hour bins to 1..4; any NaN hour gets the lowest congestion 1 (or choose a neutral 2)
    bins = [0, 9, 15, 20, 24]
    labels = [1, 2, 3, 4]
    valid_hours = df['departure_hour'].between(0, 23, inclusive='both')
    df.loc[valid_hours, 'congestion_factor'] = pd.cut(
        df.loc[valid_hours, 'departure_hour'],
        bins=bins, labels=labels, right=False, include_lowest=True
    ).astype('Int64')
    df['congestion_factor'] = df['congestion_factor'].fillna(1).astype(int)


    # Route difficulty
    df['route_difficulty_flag'] = df['scheduled_arrival_station_code'].isin(['ASE', 'GRU', 'JLN', 'FAT', 'LHR']).astype(int)


    # Total volume
    df['total_volume'] = df['total_pax'] + df['total_checked_bags']


    # Step 7: Daily normalization, scoring, ranking, classification
    features_to_normalize = [
        'turnaround_pressure', 'baggage_complexity_score', 'congestion_factor',
        'passenger_service_load', 'route_difficulty_flag', 'total_volume'
    ]


    def normalize_group(g):
        for f in features_to_normalize:
            min_v = g[f].min()
            max_v = g[f].max()
            if pd.isna(min_v) or pd.isna(max_v) or max_v == min_v:
                g[f'norm_{f}'] = 0.0
            else:
                g[f'norm_{f}'] = (g[f] - min_v) / (max_v - min_v)
        # Weighted score
        g['difficulty_score'] = (
            0.30 * g['norm_turnaround_pressure'] +
            0.25 * g['norm_baggage_complexity_score'] +
            0.15 * g['norm_congestion_factor'] +
            0.15 * g['norm_passenger_service_load'] +
            0.10 * g['norm_route_difficulty_flag'] +
            0.05 * g['norm_total_volume']
        )
        # Rank within day (1 = hardest)
        g['daily_rank'] = g['difficulty_score'].rank(method='dense', ascending=False).astype(int)
        # Quantile-based classification on scores to approximate 20/60/20 by flight count
        if len(g) >= 5:
            low_thr = g['difficulty_score'].quantile(easy_q)
            high_thr = g['difficulty_score'].quantile(difficult_q)
            g['difficulty_category'] = np.select(
                [g['difficulty_score'] >= high_thr, g['difficulty_score'] <= low_thr],
                ['Difficult', 'Easy'],
                default='Medium'
            )
        else:
            # Fallback for very small days: rank-based 1-2 hardest as Difficult, last 1-2 as Easy
            max_rank = g['daily_rank'].max()
            g['difficulty_category'] = np.where(
                g['daily_rank'] <= max(1, round(0.2 * max_rank)),
                'Difficult',
                np.where(
                    g['daily_rank'] >= max_rank - max(0, round(0.2 * max_rank) - 1),
                    'Easy',
                    'Medium'
                )
            )
        return g


    final_df = df.groupby('scheduled_departure_date_local', group_keys=False).apply(normalize_group)


    # Step 8: Output selection (include available identifiers only)
    desired_cols = [
        'company_id', 'flight_number', 'scheduled_departure_date_local',
        'scheduled_departure_station_code', 'scheduled_arrival_station_code',
        'turnaround_pressure', 'baggage_complexity_score', 'congestion_factor',
        'passenger_service_load', 'route_difficulty_flag', 'total_volume',
        'difficulty_score', 'daily_rank', 'difficulty_category'
    ]
    output_cols = [c for c in desired_cols if c in final_df.columns]

    # Post-sort by category (Difficult > Medium > Easy) and then by daily_rank, preserving tie order
    if 'difficulty_category' in final_df.columns and 'daily_rank' in final_df.columns:
        cat_order = pd.CategoricalDtype(categories=['Difficult', 'Medium', 'Easy'], ordered=True)
        final_df['_cat_key'] = final_df['difficulty_category'].astype(cat_order)
        final_df = final_df.sort_values(by=['_cat_key', 'daily_rank'], ascending=[True, True], kind='mergesort')
        final_df = final_df.drop(columns=['_cat_key'])

    return final_df[output_cols].reset_index(drop=True)




if __name__ == '__main__':
    FLIGHT_DATA_PATH = '/content/Flight Level Data.csv'
    BAG_DATA_PATH = '/content/Bag Level Data.csv'
    PNR_FLIGHT_PATH = '/content/PNR Flight Level Data.csv'
    PNR_REMARK_PATH = '/content/PNR Remark Level Data.csv'


    submission_df = calculate_flight_difficulty_score(
        FLIGHT_DATA_PATH, BAG_DATA_PATH, PNR_FLIGHT_PATH, PNR_REMARK_PATH,
        use_random_conn_time=True, difficult_q=0.8, easy_q=0.2
    )
    YOUR_NAME = 'United_Together'
    output_filename = f'test_{YOUR_NAME}.csv'
    submission_df.to_csv(output_filename, index=False)
    print(f"Successfully generated submission file: {output_filename}")
